In [1]:
# 設定
# T=120
# 道路はトラック1台が通る制約

In [2]:
import numpy as np, pandas as pd
from pulp import lpSum,LpStatus,value,LpInteger,GUROBI_CMD
from ortoolpy import addvars, addvals, model_min

Using license file C:\Users\h16m922r24\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-07
No parameters matching '_test' found


In [3]:
#モデル作成
m = model_min()

In [4]:
#セルの番号分け
list_広域 = [1,2,3]
list_民間 = [4,5,6,7,8,9,10,11,12,13,14]
list_道路 = []

for i in range(15,55):
    list_道路.append(i)

In [5]:
#滞留量のdataframe
df_x = pd.DataFrame(
    [(a, t) for a in range(1,55) for t in range(0,101)],
    columns=['a', 't'])

In [6]:
#流出量のdataframe #民間セル以外
df_v = pd.DataFrame(
    [(a, t) for a in list_広域 + list_道路 for t in range(1,101)],
    columns=['a', 't'])

In [7]:
#流入量のdataframe #広域セル以外
df_u = pd.DataFrame(
    [(a, t) for a in list_民間 + list_道路 for t in range(1,101)],
    columns=['a', 't'])

In [8]:
#セルの種類の判別
def give_cat(x): 
    if x == 1 or x == 2 or x == 3:
        return '広域'
    elif x==4 or x==5 or x==6 or x==7 or x==8 or x==9 or x==10 or x==11 or x==12 or x==13 or x==14:
        return '民間'
    else:
        return '道路'
    
df_x['cat'] = df_x['a'].apply(give_cat)
df_v['cat'] = df_v['a'].apply(give_cat)
df_u['cat'] = df_u['a'].apply(give_cat)

In [9]:
#セルごと所要時間の追加
def give_time(x):  
    if x == '民間':
        return 0
    else:
        return 5

df_x['time'] = df_x['cat'].apply(give_time)

In [10]:
#変数追加
addvars(df_x) #非負制約クリア
addvars(df_v)
addvars(df_u);

In [11]:
#目的関数
m += lpSum(df_x.Var*df_x.time)

In [12]:
#滞留量の上限
def give_cap(x):
    if x == 1:
        return 50000
    elif x == 2:
        return 50000
    elif x == 3:
        return 50000
    elif x==4 or x==5 or x==6 or x==7 or x==8 or x==9 or x==10 or x==11 or x==12 or x==13 or x==14:
        return 7600
    else:
        return 600 #トラック1台分
    
df_x['cap'] = df_x['a'].apply(give_cap)

In [13]:
#滞留量の制約
for row in df_x.itertuples():
    m += row.Var <= row.cap

In [14]:
#流出量の上限
def give_cap_v(x):
    if x == 1 or x == 2 or x == 3 :
        return 150
    else:
        return 600
df_v['cap'] = df_v['a'].apply(give_cap_v)

↓　**リフトの台数変更**　↓

In [15]:
#リフトのdataframe
df_l = pd.DataFrame(
    [1,2,3],
    columns=['a'])

addvars(df_l, cat=LpInteger)

m += lpSum(df_l.Var) == 6

df_l

,a,Var
0,1,v014855
1,2,v014856
2,3,v014857


In [16]:
#流出量の制約 リフトVer.
for row in df_v.itertuples():
    if row.a == 1 :
        m += row.Var <= row.cap * df_l.loc[0,'Var']
    elif row.a == 2 :
        m += row.Var <= row.cap * df_l.loc[1,'Var']
    elif row.a == 3 :
        m += row.Var <= row.cap * df_l.loc[2,'Var']
    else:
        m += row.Var <= row.cap

In [17]:
#流入量の上限
def give_cap_u(x):
    if x==4 or x==5 or x==6 or x==7 or x==8 or x==9 or x==10 or x==11 or x==12 or x==13 or x==14:
        return 7600
    else:
        return 600
    
df_u['cap'] = df_u['a'].apply(give_cap_u)

In [18]:
#流入量の制約
for row in df_u.itertuples():
    m += row.Var <= row.cap

In [19]:
def df_xi(x):
    return df_x.query('a == @x')

def df_vi(x):
    return df_v.query('a == @x')

def df_ui(x):
    return df_u.query('a == @x')

In [20]:
#交通量保存則
#t=0,1のときはcontinue
#df_vとdf_uはt=1から存在するので、df_xよりも1多くマイナスする

In [21]:
#広域セルの制約
for i in list_広域 :
    df_xx = df_xi(i)
    df_xx.reset_index(inplace=True,drop=True)
    df_vx = df_vi(i)
    df_vx.reset_index(inplace=True,drop=True)
    
    for i in range(len(df_xx)):
        if i == 0 or i == 1 :
            continue
        x = df_xx['Var'].iloc[i-1] - df_vx['Var'].iloc[i-1]
        m += df_xx['Var'].iloc[i] == x

In [22]:
#道路セルの制約
for i in list_道路 :
    df_xx = df_xi(i)
    df_xx.reset_index(inplace=True,drop=True)
    df_vx = df_vi(i)
    df_vx.reset_index(inplace=True,drop=True)
    df_ux = df_ui(i)
    df_ux.reset_index(inplace=True,drop=True)
    
    for i in range(len(df_xx)):
        if i == 0 or i == 1 :
            continue
        x = df_xx['Var'].iloc[i-1] - df_vx['Var'].iloc[i-1] + df_ux['Var'].iloc[i-2]
        m += df_xx['Var'].iloc[i] == x

In [23]:
#民間セルの制約
for i in list_民間 :
    df_xx = df_xi(i)
    df_xx.reset_index(inplace=True,drop=True)
    df_ux = df_ui(i)
    df_ux.reset_index(inplace=True,drop=True)
    
    for i in range(len(df_xx)):
        if i == 0 or i == 1 :
            continue
        x = df_xx['Var'].iloc[i-1] + df_ux['Var'].iloc[i-2]
        m += df_xx['Var'].iloc[i] == x

In [24]:
#流出入量の一致
node = pd.read_csv('node_data3.csv')
node['I'] = node['I(i)'].str.split(',')
node['O'] = node['O(i)'].str.split(',')
node = node.drop(['I(i)', 'O(i)'], axis=1)

for row in node.itertuples():
    for t in range(1,101):
        m += lpSum(df_v.query('a == @row.I & t == @t').Var) == lpSum(df_u.query('a == @row.O & t == @t').Var)

In [25]:
# #t=1 の物資量は合計50の物資をランダムに振り分け
start_amount = df_x.query('cat=="広域" & t == 1') 
start_amount
m += lpSum(start_amount.Var) == 83600

others = df_x.query('cat!="広域" & t==1')
for row in others.itertuples():
    m += row.Var == 0

In [26]:
#需要を満たすこと
for row in df_x.query('cat=="民間" & t == 97').itertuples():
    m += row.Var == row.cap

In [27]:
#初期条件
for row in df_x.query('t == 0').itertuples():
    m += row.Var == 0

#この式も必須    
for row in df_v.query('t == 1').itertuples():
    m += row.Var == 0

In [28]:
%time m.solve()

Wall time: 4.87 s


1

In [29]:
addvals(df_l)
addvals(df_x) 
addvals(df_v)
addvals(df_u);

In [30]:
df_l

,a,Var,Val
0,1,v014855,0.0
1,2,v014856,3.0
2,3,v014857,3.0


In [31]:
print(value(m.objective))

19623000.0


In [32]:
df_x.query('t==1 & Val> 0 ')

,a,t,cat,time,Var,cap,Val
102,2,1,広域,5,v000103,50000,41850.0
203,3,1,広域,5,v000204,50000,41750.0


In [33]:
df_x.query('cat=="広域" & t>=141')

,a,t,cat,time,Var,cap,Val


In [34]:
df_v.query('t == 2 & Val >0')

,a,t,cat,Var,cap,Val
101,2,2,広域,v005556,150,450.0
201,3,2,広域,v005656,150,450.0


In [35]:
df_u.query('t ==5 & Val >0')

,a,t,cat,Var,cap,Val
204,6,5,民間,v009959,7600,300.0
2304,27,5,道路,v012059,600,450.0
2404,28,5,道路,v012159,600,450.0
2504,29,5,道路,v012259,600,150.0
3904,43,5,道路,v013659,600,450.0
4304,47,5,道路,v014059,600,450.0
4704,51,5,道路,v014459,600,300.0
4804,52,5,道路,v014559,600,150.0


In [36]:
df_v.query('0 < Val < 1 ')

,a,t,cat,Var,cap,Val


In [37]:
df_u.query('0 < Val < 1 ')

,a,t,cat,Var,cap,Val


In [38]:
print(m)

NoName:
MINIMIZE
5*v000001 + 5*v000002 + 5*v000003 + 5*v000004 + 5*v000005 + 5*v000006 + 5*v000007 + 5*v000008 + 5*v000009 + 5*v000010 + 5*v000011 + 5*v000012 + 5*v000013 + 5*v000014 + 5*v000015 + 5*v000016 + 5*v000017 + 5*v000018 + 5*v000019 + 5*v000020 + 5*v000021 + 5*v000022 + 5*v000023 + 5*v000024 + 5*v000025 + 5*v000026 + 5*v000027 + 5*v000028 + 5*v000029 + 5*v000030 + 5*v000031 + 5*v000032 + 5*v000033 + 5*v000034 + 5*v000035 + 5*v000036 + 5*v000037 + 5*v000038 + 5*v000039 + 5*v000040 + 5*v000041 + 5*v000042 + 5*v000043 + 5*v000044 + 5*v000045 + 5*v000046 + 5*v000047 + 5*v000048 + 5*v000049 + 5*v000050 + 5*v000051 + 5*v000052 + 5*v000053 + 5*v000054 + 5*v000055 + 5*v000056 + 5*v000057 + 5*v000058 + 5*v000059 + 5*v000060 + 5*v000061 + 5*v000062 + 5*v000063 + 5*v000064 + 5*v000065 + 5*v000066 + 5*v000067 + 5*v000068 + 5*v000069 + 5*v000070 + 5*v000071 + 5*v000072 + 5*v000073 + 5*v000074 + 5*v000075 + 5*v000076 + 5*v000077 + 5*v000078 + 5*v000079 + 5*v000080 + 5*v000081 + 5*v000082 +